In [1]:
from urllib.request import urlopen, Request
import sys
print(sys.version)
import requests
from bs4 import BeautifulSoup
import lxml.html
import urllib.parse as urlparse
from urllib.parse import urljoin
import re
import string
import urllib3
import pandas as pd
import time
import os
import glob
import csv
import xlsxwriter
from itertools import chain
import re
import time

start = time.time()

#from signal import *
#import os, time

#def clean(*args):
#    print('Forced exit!')
#    os._exit(0)

#for sig in (SIGABRT, SIGINT, SIGTERM):
#    signal(sig, clean)
list1 = []
urls_file = 'TrainingOEMs.txt'
tags_file = 'tags_file.txt'
urllib3.disable_warnings()

# Open url and return page content as BeautifulSoup object
def get_dom(url):
	try:
		connection = requests.get(url, verify=False, allow_redirects=True)
		return connection
	except Exception as e: 
		print(e)
	return False

# Return links from url
def get_links(url):
	u = url
	url = get_dom(url)
	if url != False:
		tags = BeautifulSoup(url.content, 'lxml').findAll('a', {"href": True})
		href_set = {tag.get('href') for tag in tags}
		return resolve_links((href for href in href_set), u)
	else:
		return False

# Return links url ( 2 layers: all links found un url page + all links on those pages)
def get_links_2layers(url):
	list = get_links(url)
	if list != False:
		for link in list:
			list = chain(list, get_links(link))
		return list
	else:
		return False

# Guess root for relative links
def guess_root(links):
    for link in links:
        if link.find('http'):
            parsed_link = urlparse.urlparse(link)
            scheme = parsed_link.scheme + '://'
            netloc = parsed_link.netloc
            return scheme + netloc

# Transform relative link to absolute link
def resolve_links(links, url):
	root = str(url)	
	for link in links:
		link = urljoin(root, link)
		
		yield link
#read elements from different lines from fname in list
def readFile(fname):
	list = []
	with open(fname) as f:
		content = f.readlines();
		for line in content:	
			list.append(line)
	return list
	
# check if link has company name in it
def checkLink(link, root):
	found = ''
	m = re.search('www(.+?).com', root)
	if m in link:
		return True
	return False

url_content = readFile(urls_file) # file with company links
url_content.pop(0)
tag_content = readFile(tags_file) # file with tags
tag_content.pop(0) # delete first row from tag file - comment
tag_content.pop(0) # delete second row from tag file - comment
nr_layers = int(tag_content[0]) # number of layers to search in
tag_content.pop(0) # delete third row from tag list (number of layers)
tag_content.pop(0)
tag_content = list({t.lower().strip('\n') for t in tag_content}) # list with tags
print(tag_content)

workbook_name = 'OEM_results_file.xlsx'
workbook = xlsxwriter.Workbook(workbook_name)



print('open page')

index = 0
for line in url_content:
	
	row = 2
	open_page = ''.join(line); # take every link from file with company links
	
	if ('\n' in open_page):
		open_page = open_page.replace('\n', '')
		#print('page opened')
		#print("open page:" + open_page)
	try:
		company = re.split('[.]', open_page)[1] # take company name from link
	except Exception as e:
		print(e)
	Email = ('@' + company + '.com')
	tag_content.insert(0, Email)
	#print("company:" + company)
	worksheet = workbook.add_worksheet(company)
	header = [company] + tag_content
	print(header)
	worksheet.write_row('A1', tuple(header)) # write header to file
		
	
	if(nr_layers == 1): # check number of layers to search into
		print('getting links (1)')
		all_links = get_links(open_page)
	elif(nr_layers == 2):
		print('getting links (1)')
		all_links = get_links_2layers(open_page)
		#print(all_links)
	else:
		all_links = ''
		print('*****************     '.join('Empty ').join(urls_file).join(' file.'))
	
	if(all_links != False): # check if we there are links in all_links
		print(type(all_links))
		
		for (link) in all_links: 
			if ('\n' in link):
				print('---------------' + link)
				link = link.replace('\n', '')
			
			com = re.search(open_page, link)
			if not com:
				continue
				
			if link in list1:
				#print("The site was repeting and was skiped")
				#print("test" + link )
				continue	
					
			list1.append(link)

			if(link != False and link.count('.pdf') == 0 and (company in link)):
				# print("aici se verifica" + link)
				
				str_to_file = [link] # create row to insert in file
				
				try:
					print("Webpage Link: " + link)
					cont = get_dom(link).content
					if (cont == False):
						break
					pa_row('A' + str(row), tuple(str_to_file)) # write values to file
						row = row + 1
						print('Wrote to file')
						
					else:
						print('No tag found')
					#workbook.close()
					#workbook_obj = openpyxl.load_workbook(workbook_name)
					
				except Exception as e: 
					print(e)
				#print(page)start_time = time.time()
			#print('\n')
			time.sleep(0.05) 	
	print('***************************************************************************')
workbook.close()

end = time.time()
print('Total execution time (minutes): ',(end - start))

ge = BeautifulSoup(cont, 'lxml')
					for script in page('script'):
						script.decompose() # delete all script modules
					for script in page('style'):
						script.decompose() # delete all style modules
					sum = 0 # sum - counts all apparitions of tags
					for i in range(0, len(tag_content)):
						tag = tag_content[i]
						nr_apparitions = str(page).lower().count(tag)
						sum = sum + nr_apparitions
						if(nr_apparitions != 0):
							print(str(nr_apparitions) + '----' + str(tag))
						str_to_file.append(nr_apparitions)
						
							
					if(sum > 0): # if sum == 0, don't write tags to file
						worksheet.write_row('A' + str(row), tuple(str_to_file)) # write values to file
						row = row + 1
						print('Wrote to file')
						
					else:
						print('No tag found')
					#workbook.close()
					#workbook_obj = openpyxl.load_workbook(workbook_name)
					
				except Exception as e: 
					print(e)
				#print(page)start_time = time.time()
			#print('\n')
			time.sleep(0.05) 	
	print('***************************************************************************')
workbook.close()

end = time.time()
print('Total execution time (minutes): ',(end - start))




3.7.5rc1 (default, Oct  8 2019, 16:47:45) 
[GCC 9.2.1 20191008]
['global foundries', 'samsung', 'amkor', 'flipchip', 'open silicon', 'wirebond', 'verysilicon', 'tsmc', 'tower jazz', 'st semi', 'guc', 'mailto', 'esilicon', 'curatorie', '2.5d', 'semiconductor']
open page
['com', '@com.com', 'global foundries', 'samsung', 'amkor', 'flipchip', 'open silicon', 'wirebond', 'verysilicon', 'tsmc', 'tower jazz', 'st semi', 'guc', 'mailto', 'esilicon', 'curatorie', '2.5d', 'semiconductor']
getting links (1)
<class 'itertools.chain'>
Webpage Link: https://tobii.com/group/about/
No tag found
Webpage Link: https://tobii.com/group/about/corporate-governance/board-committees/
No tag found
Webpage Link: https://tobii.com/group/about/corporate-governance/auditor/
No tag found
Webpage Link: https://tobii.com/group/about/history-of-tobii/
No tag found
Webpage Link: https://tobii.com/group/about/corporate-social-responsibility/
No tag found
Webpage Link: https://tobii.com/group/search/
No tag found
Webpag

No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/mthgwo62guncq5evpxdo/TobiiDynavox-I-13_I-16-Angled-Back.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/tem69b9z0xffffu9qd9a/Anand_5000x3121px.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/ry98se8l7okiqs6yp8mc/TobiiDynavox-I-13_I-16-Jeff.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/74nfozeymg3daitznrdk/Eye_tracking_study_in_lab_Tobii_Pro_Fusion.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/mdnp5djdiec4k7001bcl/TobiiDynavox-I-13_I-16-Angie.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/isvs5jqwi1hxnwci1p5k/Eye_tracking_school_Tobii_Pro_Fusion.jpg?download=1
No tag found
Webpage Link: https://tobii.com/imagevault/publishedmedia/a24085iejemkva400rtq/Tobii_Pro_Fusion_screen_based_eye_tracker_scientif.jpg?download=1
No tag fou

No tag found
Webpage Link: https://tobii.com/#ContentArea1
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/2018/silent-period-q3-2018/
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/2019/interim-report-q3-2019/
1----mailto
Wrote to file
Webpage Link: https://tobii.com/group/investors/calendar/2019/roadshow-with-abg-sundal-collier/
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/2019/carnegie-and-abg-sundal-collier-seminar/
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/2019/interim-report-q2-2018/
1----mailto
Wrote to file
Webpage Link: https://tobii.com/group/investors/calendar/2018/carnegie-small-mid-cap-seminar/
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/2019/interim-report-q1-2019/
1----mailto
Wrote to file
Webpage Link: https://tobii.com/group/investors/calendar/#/
No tag found
Webpage Link: https://tobii.com/group/investors/calendar/#OverlayMenu
No tag found
Webpag

2----mailto
Wrote to file
Webpage Link: https://tobii.com/group/news-media/#OverlayMenu
2----mailto
Wrote to file
Webpage Link: https://tobii.com/group/about/board-of-directors/#/
No tag found
Webpage Link: https://tobii.com/group/about/board-of-directors/#OverlayMenu
No tag found
Webpage Link: https://tobii.com/group/about/diversity/#/
1----mailto
Wrote to file
Webpage Link: https://tobii.com/group/about/diversity/#OverlayMenu
1----mailto
Wrote to file
***************************************************************************
Total execution time (minutes):  628.7898285388947
